<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_XLNet/test_sample_XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install datasets
! pip install sentencepiece

In [ ]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/modeling_XLNet/

In [3]:
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import XLNetTokenizer
from model import XLNetClassify

In [ ]:
dataset = load_dataset('tweets_hate_speech_detection')

In [5]:
sentences = []
labels = []
for data in dataset['train']:
  sentences.append(data['tweet'])
  labels.append(data['label'])
  
assert len(sentences) == len(labels)
print('Total Samples: ', len(sentences))

Total Samples:  31962


In [11]:
class CustomDataset(Dataset):
  def __init__(self, tokenizer, list_sentences, labels=None, max_len=16):
    self.tokenizer = tokenizer
    self.list_sentences = list_sentences
    self.labels = labels
    self.max_len = max_len
  
  def __len__(self):
    return len(self.list_sentences)
  
  def __getitem__(self, idx):
    texts = self.list_sentences[idx]
    tokens = tokenizer(texts, max_length=self.max_len,
                        padding='max_length', truncation=True,
                        return_tensors='pt')
    if self.labels is not None:
      tgt = torch.tensor(self.labels[idx])
    else:
      tgt = None
    
    return {
        'inp_ids': tokens['input_ids'],
        'inp_mask': tokens['attention_mask'],
        'tgt_ids' : tgt,
    }

In [7]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

In [14]:
sample_sentences = sentences[:100]
sample_labels = labels[:100]
sample_dataset = CustomDataset(tokenizer,
                               list_sentences=sample_sentences,
                               labels=sample_labels,
                               max_len=16)
sample_loader = DataLoader(dataset=sample_dataset,
                           batch_size=2,
                           shuffle=False)
for sample in sample_loader:
  assert sample['inp_ids'].squeeze(1).dim() == 2
  assert sample['inp_ids'].squeeze(1).size(1) == sample['inp_mask'].squeeze(1).size(1)
  assert sample['tgt_ids'].size(0) == sample['inp_ids'].size(0)
  break